# Table Manipulation

## Reading

[Chapter 8: 8.2 - 8.4](https://inferentialthinking.com/chapters/08/Functions_and_Tables.html)



In this notebook we explore more advanced methods to work with data in a table or DataFrame.

First we'll learn to write functions that can help us work more efficiently, then we learn how to build and reshape a DataFrame, and how to separate data into groups to evaluate them.

---

## Writing Functions
Functions help us work more efficiently with the code that we use for data analysis.

As shown in the textbook, suppose we have a dataset of the heights of parents and children in different families. We want to use the dataset to predict a child's height, given the parents' heights.

First, we read in the height data into a table or DataFrame, and then we inspect the data.

In [ ]:
import pandas as pd

url = "https://raw.githubusercontent.com/DeAnzaDataScience/CIS11/refs/heads/main/datasets_notes/galton.csv"
family = pd.read_csv(url)
print("First 5 rows:")
family.head()

Each row is for one child in the family, and the columns that we're interested in are the heights of the `father`, the `mother`, and the `childHeight`.

Next, we would like to predict the child's height based on the parents' average heights.

<u>Step 1</u>:<br>
We select the parents' height columns, find the average of the parents' height, and create a DataFrame of the heights.


In [ ]:
# find the average of the parents' height
avg = (family.father + family.mother) / 2

# create a new DataFrame with the avg and childHeight data
heights = pd.DataFrame({'avg': avg,
                        'child': family.childHeight})
print("First 5 rows:")
heights.head()

<u>Step 2</u>:<br>
We use a scatterplot to see if there's correlation between the parents' average height and the child's height.

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(heights.avg, heights.child, alpha=0.5)
plt.title("Child Height vs. Parent Average Height")
plt.xlabel("Parent Average Height")
plt.ylabel('Child Height')
plt.grid()
plt.show()

It looks like there's positive correlation: in general, the taller the parents, the taller the child. Because there is correlation, we can use the parents' data to predict the child's height.

Suppose we want to predict the child's height, given that the parents' average height is 68 inches.

<u>Step 3</u>:<br>
From the dataset, we:
1. Extract all the average parents' heights that are within 1/2 inch from 68 inches, which is the range of 67.5 to 68.5 inches.
2. Use these parent height averages to look up the corresponding child's heights, and find the mean of all the child heights.

The resulting mean will be our predicted child height.

In [ ]:
import numpy as np

# 1. Find all the parent height averages in the range around 68 inches
# and create a new DataFrame with them
parent_range = heights[(heights.avg >= (68-0.5)) & (heights.avg < (68+0.5))]

# 2. Find the mean of all the corresponding child heights
child_height = np.mean(parent_range.child)

# 3. Print the child's height
print("Predicted child's height:", round(child_height, 2))

The method of averaging a range of data that are close to our target data (68 inches) and use the average to predict the child's height is called the _nearest neighbor_ prediction. This method is the foundation in many machine learning algorithms that are used for predictions.

Now we want to make the above prediction calculation for _any_ parent average height. This means we want to find a way to conveniently run the lines of code above, without having to retype them for each prediction. The convenient way is to put the 3 lines of code into a _function_.

<u>Step 4</u>:<br>
Generalize the prediction method by _writing a function_ to use with any parent average height.

Recall that we first worked with functions in Week 2, where we used or _called_ functions such as round() or print(). A function is a block of code that has a name and performs a task. The print() function has the name _print_ and its job is to print all the data that are put in between the parentheses `( )`.

We now learn to write a function or the block of code so that will do the work that we want. Then we call our function in the same way that we call print().

To write our own function we use the following format:

```
def function_name(input to function):
    steps to do a task
    return result from the task
```

- The keyword `def` is short for _definition_. A function definition is the block of code that does the task of the function.
- The function_name can be any word(s) that describes what task the function does.
- The `(input to function)` are the variables that store the input to the function.
- The first line of the function definition ends with colon `:`. This line is called the _function header_.
- The block of code that performs the function's task is called the _function body_. It is indented from the function header.
- If the function's task produces new data or result, the result is `return`ed as output from the function.

In the Code cell above, we notice that our task to predict a child's height is made up of 3 steps. Therefore, putting together the function definition format and the 3 lines of code that make up the function's task, we have:


In [ ]:
# The function name is predict_child
# The input is the parent height average
def predict_child(parent_avg):

  # 1. find the parent height averages within the 1/2 inch range
  parent_range = heights[(heights.avg >= (parent_avg-0.5)) &
                         (heights.avg < (parent_avg+0.5))]
  # 2. find the child's height
  child_avg = np.mean(parent_range.child)
  # 3. return the child's height
  return np.round(child_avg, 2)

Comparing the 2 lines of code where we calculated the predicted child height from the parent average of 68:
```
parent_range = heights[(heights.avg >= (68-0.5)) & (heights.avg < (68+0.5))]
child_height = np.mean(parent_range.child)
print("Child's height:", round(child_height, 2))
```
We notice that there are 2 changes that we made in the original code:
- substitute the value `68` with the variable `parent_avg`
- instead of printing the child's height, we `return` the child's height.

To run the function with a parent height average as input, we use the format:<br>
`returned_data = function_name(input data)`

In [ ]:
child_height = predict_child(68)
print("Parent:", 68, "Child:", child_height)

child_height = predict_child(66)
print("Parent:", 66, "Child:",child_height)

Note that after the `predict_child` function is written, we can run or call it any time we want to predict a child's height, we don't need to repeat the 3 calculation steps each time.

The advantage of writing _functions_ are:
- Avoid repetition: we don't need to copy and paste a block of code over and over each time we want to do the same task.
- Easier to read the code: it's easier to recognize what `predict_child` would do, compared to reading the 3 lines of code that make up this function.
- Easier to maintain the code: if we decided to change the calculation to predict the child's age, we would only need to make the change in the `predict_child` function, instead of searching for the 3 lines of code and change them throughout the notebook.

---

## Grouping Data by a Feature

Often data scientists need to divide the dataset into groups to inspect the data in each group or to compare the groups.

For example, if the dataset contains rows of information about the ice cream that people buy at a store:
- what time of year: spring, summer, fall, winter
- the amount of ice cream: how many scoops
- the ice cream flavor: chocolate, strawberry, vanilla
- the price: how much per scoop

If we want to find the data patterns for a specific flavor of ice cream, we can save all the rows of data for chocolate into a group, and all the rows for strawberry and for vanilla into two other groups. With these groups, we can look for the popular time of year for chocolate vs. strawberry vs. vanilla, or what's the total amount of money people spend on a specific ice cream flavor.

To form groups based on a data feature, such as the ice cream flavor, the feature must be _categorical data_.

Recall that categorical data means that the data must be its own distinct category and not part of a continuous range. In the ice cream example, the time of year (the 4 seasons) and the ice cream flavor (3 flavors) are categorical data. They are not part of a continuous range of data.

Can you tell which feature of the ice cream dataset is not categorical data?

We now use a dataset to explore how to form groups of data by a feature. This dataset is used in the book and is a dataset of NBA player salaries, which we read in into a DataFrame:

In [ ]:
import pandas as pd

url = 'https://raw.githubusercontent.com/DeAnzaDataScience/CIS11/refs/heads/main/datasets_notes/nba-salaries.csv'
nba = pd.read_csv(url)
nba

We want to look at the latest season only, so we first create a DataFrame where the season is 2020.

In [ ]:
latest_nba = nba[nba.season == 2020]
latest_nba.head()

In the dataset above, the `position` and `team` are the categorical data.

To form groups based on a feature, pandas provides us with a `groupby` method and several options such as: `count`, `sum`, `max`, `min`



### Count of groups
To group the data by a feature and find the total count of each category of feature, we use the format:<br>
`output = a_DataFrame.groupby('feature').a_column.count()`

The groupby method returns each group by `feature`, complete with all the columns except the feature column, so we need to choose a colum and then we ask for the count of that column. We can choose any column since the count is the same for each column.

In [ ]:
# group by position and find the count of each position
latest_nba.groupby('position').name.count()

In [ ]:
# Can you write code to print each team name and the count of each team?
# Hint: group the data by team name


### Basic Statistics of Groups
To group the data by a feature and find the statistics of a certain column in each group, we use the format:<br>
`output = a_DataFrame.groupby('feature').a_column.statistical_function()`

The groupby method returns each group by feature, complete with all the columns except the feature column, so we need to choose the column that we want and ask for the specific statistics, some of which are:
- sum()
- max()
- min()
- mean()
- median()

In [ ]:
# find the mean salary of each position
avg_salary = latest_nba.groupby('position').salary.mean()
print("Salary in millions")
round(avg_salary/1000000,2)

In [ ]:
# Can you find how much each team pays all of its players?


### Grouping by Two Features
To group data by more than one features, we simply give the `groupby`method the features we want. We do this by listing the features inside `[]`, which you may recall is the format of a list in Python.

In [ ]:
# show each position, and with each position show all the teams
# along with the count of that position in the team
latest_nba.groupby(['position', 'team']).name.count()

Separating data into groups is useful when we need to compare the groups or do calculations on groups of data.

---

## Pivoting
Sometimes it is useful to reshape the data in a table in order to see the patterns in the data more clearly. When we select certain columns (or features of the data) and display them against each other, this is called _pivoting_ data.

To pivot, we use 3 columns of the DataFrame as the key components of a pivot:
- One column will stay the same, with rows of data, and serves as the _index_ of the pivot.
The data must be categorical data.
- One column will be separated into multiple columns and serves as the _columns_ of the pivot. The separated column labels are the unique categories of data, and the data must be categorical data.
- One column will be used to fill in the new  table, and serve as the _values_ of the pivot.

Optionally we can also specify the _aggfunc_, which tells the pivot method to do some basic calculation with the _values_ data. The basic calculation can be finding the sum, the mean, the count, the max, or the min of the data.



Suppose we have a dataset of the 2 types of drinks and their popularity ranking during 3 years

In [ ]:
drinks = pd.DataFrame({'drink': ['coffee', 'tea', 'coffee', 'tea', 'coffee', 'tea'],
                      'year': [2023, 2023, 2022, 2022, 2021, 2021],
                      'ranking': [1, 2, 2, 1, 1, 2]})
drinks

With the table above it's not easy to see which drink is more popular during the 3 years. To make it easier to see the data, we pivot the table by keeping the same `drink` column, but changing the `year` column into separate columns. This will make it easier to see the ranking.

We use the `pivot` method and we need to give 3 input to the method:
- The `index` input will be the `drink` column.
- The `columns` input will be the `year` column.
- The `values` input will be the `rank` column.

In [ ]:
drinks.pivot(index='drink', columns='year', values='ranking')

Now it's easier to see that coffee is more popular than tea since it's ranked first in 2 of the 3 years.

We can also pivot the other way by swapping the `drink` and `year` in the new table. This new table will also show more clearly that coffee is the popular drink than the original table could.

In [ ]:
drinks.pivot(index='year', columns='drink', values='ranking')

Next we look at a larger dataset which is introduced in the textbook. The dataset is about personal income and education level for the US population from 2008 to 2014.

In [ ]:
url = 'https://raw.githubusercontent.com/DeAnzaDataScience/CIS11/refs/heads/main/datasets_notes/educ_inc.csv'
salary = pd.read_csv(url)
salary

We're interested in the `Educational Attainment`, `Personal Income`, and `Population Count` columns, so we create a new DataFrame with these 3 columns.

In [ ]:
salary_education = pd.DataFrame({'Education': salary['Educational Attainment'], 
                                 'Income': salary['Personal Income'], 
                                 'Population' : salary['Population Count']})
salary_education.head()

Since we will look at the total population count between 2008 and 2014, which is a large number, we will scale down the population so it's in units of 1 million.

In [ ]:
salary_education.Population = salary_education.Population / 1000000
salary_education.head()

From the table it's not easy to see the income vs education level, so we pivot the data by showing a table with rows of `Income` and columns of `Educational`. In addition, we want to sum up the total number of people for each category, so we give the `aggfunc` input the `sum` string.

In [ ]:
totals = salary_education.pivot_table(index='Income',
                               columns='Education',
                               values='Population',
                               aggfunc='sum')
totals

We now see that for the lowest income, level A, the largest population has no high school diploma. And for the highest income, level H, the largest population has a Bachelor's degree or higher.

We can also put our data visualization skills to work and plot the population count for the lowest and highest income level:

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(4,4))
plt.barh(totals.index, totals.iloc[:,0])
plt.title("Bachelor's Degree or Higher")
plt.xlabel("Population (millions)")
plt.grid()
plt.plot()

In [ ]:
plt.figure(figsize=(4,4))
plt.barh(totals.index, totals.iloc[:,3])
plt.title("No High School Diploma")
plt.xlabel("Population (millions)")
plt.grid()
plt.plot()

The plots show the income advantage of a college degree.

---

## Joining
Sometimes the data we need are stored in different tables. Each table shares some user-related data, but each table also stores some unique user data.

An example would be when a traveler books a trip by using a travel aggregator (like Expedia) to buy their airline ticket and hotel.
- The travel aggregator table will store the traveler's ID, name, email, booking date, flight id, hotel id, and payment.
- The airline table will store the traveler's airline ID, name, email, flight number, departure airport, arrival airport, baggage status.
- The hotel table will store the traveler's hotel ID, name, email, room type, date and length of stay.

The data for the trip are across 3 different tab;es, but they share the traveler's name and email.

In this case we need to combine or _join_ the 3 tables into one table before we can analyze the data.




To see how joining 2 tables work, we use the same `drinks` dataset of coffee and tea that we used in the pivot section above.

In [ ]:
drinks

Suppose we have another table that shows the price of the coffee and tea as well as an extra drink, soda.

In [ ]:
prices = pd.DataFrame({'drink': ['coffee', 'tea','soda'],
                       'price': [3.5, 2.5, 1.0]})
prices

We can see that the `drink` column is where the 2 tables have similar data.

We join the `drinks` and `prices` table by using the format:
`table1.join(table2.set_index(common_column), on = common_column)`

With this format we add table2 into table1. Both tables need to specify the common column between the 2 tables.

In [ ]:
drinks.join(prices.set_index('drink'), on ='drink')

Now we have a table with data from both `drinks` and `prices`. Note that:
- Even though `coffee` appears one time in `prices`, it has been copied into each of the `coffee` rows in `drinks`.
- The `soda` row doesn't appear in the final table since it's not in the `drinks` table.

---

In this notebook we learn how to combine code together into functions, and how to combine data together in groups, pivot table, and new DataFrames. We now have enough experience with programming tools, and in the next notebooks we'll apply them in statistical methods that help us analyze data.